In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [44]:
## upload the trained dataset , model and scaler , label encoder
from tensorflow.keras.models import load_model

model = load_model("model.h5")

## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encoder.pkl",'rb') as file:
    label_encoder_geo = pickle.load(file)

    
with open("scaler.pkl",'rb') as file:
    scaler = pickle.load(file)

In [45]:
### based on this data we will predict the churn
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France', # i have to convert this value into numeric
    'Gender': 'Male',      # i have to convert this value into numeric
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

# we will scale down the data and convert categorical data into numeric using label encoder

In [49]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
label_encoder_geo.get_feature_names_out(['Geography'])


d:\udemy generative AI\9- ANN project Implementation\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [60]:
geo_encoded

array([[1., 0., 0.]])

In [52]:
input_df = pd.DataFrame([input_data])

In [53]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [50]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [56]:
## Encoding categorical variable 'Gender'
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [58]:
input_df = input_df.drop('gender',axis=1)

In [59]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [61]:
## concatination one hot encoded geography columns with input data frame 
input_df = pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [62]:
### Scale the input data
input_df_scaled = scaler.transform(input_df)

In [63]:
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [64]:
## predict the churn
prediction = model.predict(input_df_scaled)

1/1 [==============================] - 1s 846ms/step


In [65]:
prediction

array([[0.02627304]], dtype=float32)

In [66]:
prediction[0]

array([0.02627304], dtype=float32)

In [67]:
prediction[0][0]

0.026273044

In [68]:
pred = prediction[0][0]

In [69]:
if pred > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
